In [1]:
import numpy as np
import pandas as pd
#import xgboost
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import time
import collections
import itertools as it
from sklearn import ensemble, linear_model
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)


### "simple" setup using 30 random agents and simulating 
#### (next section is fitting a strategy given the random choice of 29 agents)

In [2]:
# Load validation data
val = pd.read_csv('validation.csv')

In [3]:
# Load CTR estimators
CTR_models_val = pd.DataFrame(np.vstack([np.loadtxt('CTR_models/logr/r1_sams_logr_6.csv'),
                                        np.loadtxt('CTR_models/gbm_dam/pCTR_gbm_dam_val.csv'), 
                                        pd.read_csv('CTR_models/gbm2/pCTR_gbm2_val.csv', index_col=0).values.flatten(), 
                                        np.loadtxt('ensemble/ensemble_base_learners_set2/val/r7_mlp_nn_14.csv')]).T,
                            columns=['log_reg', 'Damian', 'Stefan2', 'NN'])

# Build ensemble
weighted_CTR_val = CTR_models_val[['Damian', 'Stefan2', 'NN']].copy()

w = np.atleast_2d([1, 6, 5]) #np.array([x1, x2, x3])
weighted_CTR_val = np.log(weighted_CTR_val/(1 + weighted_CTR_val))
weighted_CTR_val =  np.dot(weighted_CTR_val, w.T)
weighted_CTR_val /= np.sum(w) # normalize weights
weighted_CTR_val = 1 / (1 + np.exp(-weighted_CTR_val)) # take logistic / sigmoid
weighted_CTR_val = weighted_CTR_val.flatten()

CTR_models_val['ensemble1'] = weighted_CTR_val
CTR_models_val.head()



,log_reg,Damian,Stefan2,NN,ensemble1
0,0.000533,0.000078,0.000291,0.000155,0.000200
1,0.000205,0.000093,0.000211,0.000017,0.000069
2,0.000139,0.000063,0.000351,0.000011,0.000072
3,0.000210,0.000075,0.000274,0.000025,0.000091
4,0.000306,0.000092,0.000335,0.000007,0.000061


In [127]:
# Define bid look-up-table
bid_look_up = pd.DataFrame()
#bid_look_up['new'] = np.zeros(10)
#bid_look_up

In [128]:
# Fit and save linear strategies

def fit_linear_strategy(pCTR_val):
    
    CTR_frac = pCTR_val / np.mean(pCTR_val)
    #print(CTR_frac.shape)
    df_sub = val[['click','payprice']].copy()
    
    start = time.time()
    clicks = []
    budget = 6250 * 1000
    for base_bid in range(200):
        #bid = 22

        bid = base_bid * CTR_frac 

        #df = val[val['payprice'] < bid][['click','payprice']]
        df = df_sub[df_sub['payprice'] < bid]
        
        cost = df['payprice'].cumsum()
        #print(eligible)
        eligible = df[cost < budget]
        #print(eligible)

        #print([base_bid, eligible['click'].sum(), eligible['payprice'].sum()])

        clicks.append([base_bid, eligible['click'].sum(), eligible['payprice'].sum()])

    #print(time.time()-start)   
    #print(np.argmax(np.array(clicks)[:,1]))
    
    opt = np.argmax(np.array(clicks)[:,1])
    
    return opt


def save_linear_strategy(bid_look_up, CTR_models_val):
    for col in CTR_models_val.columns:

        #if col == 'log_reg':

        pCTR_val = CTR_models_val.loc[:,col]
        base_bid = fit_linear_strategy(pCTR_val)
        
        CTR_frac = pCTR_val / np.mean(pCTR_val)
        bid_look_up[col + '_linbid'] = np.round(base_bid * CTR_frac)
        
    return bid_look_up

bid_look_up = save_linear_strategy(bid_look_up, CTR_models_val)

In [129]:
# Fit and save linear strategies

def fit_rev_linear_strategy(pCTR_val):
    
    CTR_frac = np.mean(pCTR_val) / pCTR_val
    #print(CTR_frac.shape)

    df_sub = val[['click','payprice']].copy()
    
    start = time.time()
    clicks = []
    budget = 6250 * 1000
    for base_bid in range(200):
        #bid = 22

        bid = base_bid * CTR_frac 

        #df = val[val['payprice'] < bid][['click','payprice']]
        df = df_sub[df_sub['payprice'] < bid]

        cost = df['payprice'].cumsum()
        #print(eligible)
        eligible = df[cost < budget]
        #print(eligible)

        #print([base_bid, eligible['click'].sum(), eligible['payprice'].sum()])

        clicks.append([base_bid, eligible['click'].sum(), eligible['payprice'].sum()])

    #print(time.time()-start)   
    #print(np.argmax(np.array(clicks)[:,1]))
    
    opt = np.argmax(np.array(clicks)[:,1])
    
    return opt


def save_rev_linear_strategy(bid_look_up, CTR_models_val):
    for col in CTR_models_val.columns:

        #if col == 'log_reg':

        pCTR_val = CTR_models_val.loc[:,col]
        base_bid = fit_rev_linear_strategy(pCTR_val)
        
        CTR_frac = np.mean(pCTR_val) / pCTR_val
        bid_look_up[col + '_revlinbid'] = np.round(base_bid * CTR_frac)
        
    return bid_look_up

bid_look_up = save_rev_linear_strategy(bid_look_up, CTR_models_val)

In [130]:
bid_look_up.sort_values(by='log_reg_linbid', ascending=False).head()

,log_reg_linbid,Damian_linbid,Stefan2_linbid,NN_linbid,ensemble1_linbid,log_reg_revlinbid,Damian_revlinbid,Stefan2_revlinbid,NN_revlinbid,ensemble1_revlinbid
38982,101508.0,347.0,25101.0,3907.0,8178.0,0.0,50.0,1.0,5.0,2.0
101057,95804.0,621.0,15568.0,5123.0,7819.0,0.0,28.0,1.0,4.0,2.0
153755,90635.0,461.0,19988.0,4980.0,8376.0,0.0,38.0,1.0,4.0,2.0
192338,86375.0,6941.0,11356.0,983.0,4317.0,0.0,2.0,1.0,19.0,3.0
86332,75781.0,3824.0,3200.0,1669.0,2835.0,0.0,5.0,5.0,11.0,5.0


In [133]:
bid_look_up.head()

,log_reg_linbid,Damian_linbid,Stefan2_linbid,NN_linbid,ensemble1_linbid,log_reg_revlinbid,Damian_revlinbid,Stefan2_revlinbid,NN_revlinbid,ensemble1_revlinbid
0,62.0,12.0,35.0,19.0,30.0,255.0,1396.0,416.0,982.0,459.0
1,24.0,15.0,25.0,2.0,10.0,661.0,1182.0,573.0,8811.0,1326.0
2,16.0,10.0,42.0,1.0,11.0,980.0,1749.0,344.0,13720.0,1276.0
3,24.0,12.0,33.0,3.0,14.0,647.0,1452.0,440.0,5973.0,1006.0
4,35.0,14.0,40.0,1.0,9.0,444.0,1193.0,361.0,21040.0,1513.0


In [136]:
# Fit and save ORTB1
# Fit and save capped linear strategies

def fit_ortb1(pCTR_val, maxbid, cc):
    
    start = time.time()
    res = []
    budget = 6250 * 1000
    df_sub = val[['click','payprice']].copy()

    for lmbda in np.linspace(10**(-8), 10**(-5),400): #[1e-8]

        #cc = 31
        lamb = lmbda #5.2*10**(-7)
        bid = np.minimum(np.sqrt(cc/lamb * pCTR_val + cc**2) - cc, maxbid)
        df = df_sub[df_sub['payprice'] < bid]
        cost = df['payprice'].cumsum()
        eligible = df[cost < budget]
        y = eligible['click'].sum() # "clicks"
        res.append([lmbda, y])

    #print(res)
    #print(np.argmax(np.array(res)[:,1]))
    
    opt = np.array(res)[np.argmax(np.array(res)[:,1]),0]
    #print(opt)
    
    return opt


def save_ortb1(bid_look_up, CTR_models_val, maxbid, cc):
    
    for col in CTR_models_val.columns:
        pCTR_val = CTR_models_val.loc[:,col]
        lamb = fit_ortb1(pCTR_val, maxbid, cc)
        #CTR_frac = pCTR_val / np.mean(pCTR_val)
        bid = np.sqrt(cc/lamb * pCTR_val + cc**2) - cc
        bid_look_up[col + '_ortb1'] = np.round(np.minimum(bid, maxbid))
    return bid_look_up

bid_look_up = save_ortb1(bid_look_up, CTR_models_val, maxbid=10**7, cc = 31)

In [137]:
bid_look_up.describe()

,log_reg_linbid,Damian_linbid,Stefan2_linbid,NN_linbid,ensemble1_linbid,log_reg_revlinbid,Damian_revlinbid,Stefan2_revlinbid,NN_revlinbid,ensemble1_revlinbid,log_reg_ortb1,Damian_ortb1,Stefan2_ortb1,NN_ortb1,ensemble1_ortb1
count,303925.000000,303925.000000,303925.000000,303925.000000,303925.000000,3.039250e+05,303925.000000,303925.000000,3.039250e+05,303925.000000,303925.000000,303925.000000,303925.000000,303925.000000,303925.000000
mean,85.999039,112.999355,86.000471,100.998523,84.999997,1.142597e+03,575.280309,335.563504,4.372314e+03,473.295247,65.836973,71.048795,69.625921,69.892770,70.441112
std,487.782326,1526.586838,509.008216,1255.130110,366.012760,8.488211e+03,504.374206,204.273215,3.234995e+04,583.728922,61.423295,84.415970,55.980105,91.749856,58.568786
min,0.000000,7.000000,5.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,14.000000,9.000000,0.000000,1.000000
25%,22.000000,18.000000,33.000000,9.000000,25.000000,1.890000e+02,190.000000,191.000000,2.160000e+02,159.000000,33.000000,29.000000,44.000000,19.000000,37.000000
50%,45.000000,52.000000,47.000000,40.000000,49.000000,3.460000e+02,335.000000,304.000000,4.770000e+02,278.000000,55.000000,61.000000,57.000000,58.000000,60.000000
75%,83.000000,91.000000,76.000000,88.000000,86.000000,7.230000e+02,973.000000,442.000000,2.233000e+03,555.000000,82.000000,89.000000,78.000000,97.000000,87.000000
max,101508.000000,144531.000000,80783.000000,103837.000000,40914.000000,1.439609e+06,2328.000000,3210.000000,8.994396e+06,34170.000000,3762.000000,4574.000000,3368.000000,4227.000000,2447.000000


In [138]:
# Fit and save ORTB2

def fit_ortb2(pCTR_val, maxbid, cc):
    
    start = time.time()
    res = []
    budget = 6250 * 1000
    df_sub = val[['click','payprice']].copy()

    for lmbda in np.linspace(10**(-8), 10**(-5),400): #[1e-8]

        #cc = 31
        lamb = lmbda #5.2*10**(-7)
        bid = np.minimum(cc * (
                          (pCTR_val + np.sqrt(cc**2*lamb**2+pCTR_val**2)/(cc*lamb))**(1/3) 
                          - (cc*lamb/(pCTR_val + np.sqrt(cc**2*lamb**2+pCTR_val**2)))**(1/3)
                         ), 
                         maxbid)

        df = df_sub[df_sub['payprice'] < bid]
        cost = df['payprice'].cumsum()
        eligible = df[cost < budget]
        y = eligible['click'].sum() # "clicks"
        res.append([lmbda, y])

    #print(res)
    #print(np.argmax(np.array(res)[:,1]))
    
    opt = np.array(res)[np.argmax(np.array(res)[:,1]),0]
    #print(opt)
    
    return opt


def save_ortb2(bid_look_up, CTR_models_val, maxbid, cc):
    
    for col in CTR_models_val.columns:
        pCTR_val = CTR_models_val.loc[:,col]
        lamb = fit_ortb2(pCTR_val, maxbid, cc)
        bid = cc * ((pCTR_val + np.sqrt(cc**2*lamb**2+pCTR_val**2)/(cc*lamb))**(1/3) - (cc*lamb/(pCTR_val + np.sqrt(cc**2*lamb**2+pCTR_val**2)))**(1/3))
        bid_look_up[col + '_ortb2'] = np.round(np.minimum(bid, maxbid))
    return bid_look_up

bid_look_up = save_ortb2(bid_look_up, CTR_models_val, maxbid=10**7, cc = 58)

In [139]:
bid_look_up.describe()
bid_look_up.head()

,log_reg_linbid,Damian_linbid,Stefan2_linbid,NN_linbid,ensemble1_linbid,log_reg_revlinbid,Damian_revlinbid,Stefan2_revlinbid,NN_revlinbid,ensemble1_revlinbid,log_reg_ortb1,Damian_ortb1,Stefan2_ortb1,NN_ortb1,ensemble1_ortb1,log_reg_ortb2,Damian_ortb2,Stefan2_ortb2,NN_ortb2,ensemble1_ortb2
0,62.0,12.0,35.0,19.0,30.0,255.0,1396.0,416.0,982.0,459.0,68.0,22.0,46.0,35.0,43.0,70.0,23.0,48.0,39.0,46.0
1,24.0,15.0,25.0,2.0,10.0,661.0,1182.0,573.0,8811.0,1326.0,35.0,25.0,37.0,6.0,19.0,37.0,27.0,38.0,4.0,18.0
2,16.0,10.0,42.0,1.0,11.0,980.0,1749.0,344.0,13720.0,1276.0,26.0,18.0,52.0,4.0,20.0,26.0,18.0,55.0,3.0,19.0
3,24.0,12.0,33.0,3.0,14.0,647.0,1452.0,440.0,5973.0,1006.0,35.0,21.0,44.0,8.0,24.0,37.0,22.0,46.0,7.0,24.0
4,35.0,14.0,40.0,1.0,9.0,444.0,1193.0,361.0,21040.0,1513.0,46.0,25.0,51.0,2.0,17.0,49.0,27.0,53.0,2.0,16.0


In [140]:
# Add capped versions of all previous strategies (linear, rev linear, ortb1, ortb2)

def cap_strategy(bid_look_up, maxbid):
    for col in bid_look_up.columns:
        bid_look_up[col + '_cap1'] = np.round(np.minimum(bid_look_up.loc[:,col], maxbid[0]))
        bid_look_up[col + '_cap2'] = np.round(np.minimum(bid_look_up.loc[:,col], maxbid[1]))
        
    return bid_look_up
        
bid_look_up = cap_strategy(bid_look_up, maxbid=[500,1000])

In [141]:
#bid_look_up.describe()
bid_look_up.head()

,log_reg_linbid,Damian_linbid,Stefan2_linbid,NN_linbid,ensemble1_linbid,log_reg_revlinbid,Damian_revlinbid,Stefan2_revlinbid,NN_revlinbid,ensemble1_revlinbid,...,log_reg_ortb2_cap1,log_reg_ortb2_cap2,Damian_ortb2_cap1,Damian_ortb2_cap2,Stefan2_ortb2_cap1,Stefan2_ortb2_cap2,NN_ortb2_cap1,NN_ortb2_cap2,ensemble1_ortb2_cap1,ensemble1_ortb2_cap2
0,62.0,12.0,35.0,19.0,30.0,255.0,1396.0,416.0,982.0,459.0,...,70.0,70.0,23.0,23.0,48.0,48.0,39.0,39.0,46.0,46.0
1,24.0,15.0,25.0,2.0,10.0,661.0,1182.0,573.0,8811.0,1326.0,...,37.0,37.0,27.0,27.0,38.0,38.0,4.0,4.0,18.0,18.0
2,16.0,10.0,42.0,1.0,11.0,980.0,1749.0,344.0,13720.0,1276.0,...,26.0,26.0,18.0,18.0,55.0,55.0,3.0,3.0,19.0,19.0
3,24.0,12.0,33.0,3.0,14.0,647.0,1452.0,440.0,5973.0,1006.0,...,37.0,37.0,22.0,22.0,46.0,46.0,7.0,7.0,24.0,24.0
4,35.0,14.0,40.0,1.0,9.0,444.0,1193.0,361.0,21040.0,1513.0,...,49.0,49.0,27.0,27.0,53.0,53.0,2.0,2.0,16.0,16.0


In [142]:
bid_look_up.describe()

,log_reg_linbid,Damian_linbid,Stefan2_linbid,NN_linbid,ensemble1_linbid,log_reg_revlinbid,Damian_revlinbid,Stefan2_revlinbid,NN_revlinbid,ensemble1_revlinbid,...,log_reg_ortb2_cap1,log_reg_ortb2_cap2,Damian_ortb2_cap1,Damian_ortb2_cap2,Stefan2_ortb2_cap1,Stefan2_ortb2_cap2,NN_ortb2_cap1,NN_ortb2_cap2,ensemble1_ortb2_cap1,ensemble1_ortb2_cap2
count,303925.000000,303925.000000,303925.000000,303925.000000,303925.000000,3.039250e+05,303925.000000,303925.000000,3.039250e+05,303925.000000,...,303925.000000,303925.000000,303925.000000,303925.000000,303925.000000,303925.000000,303925.000000,303925.000000,303925.000000,303925.000000
mean,85.999039,112.999355,86.000471,100.998523,84.999997,1.142597e+03,575.280309,335.563504,4.372314e+03,473.295247,...,62.435844,62.466138,69.643122,69.840961,67.016675,67.061022,66.195045,66.396147,67.649101,67.680740
std,487.782326,1526.586838,509.008216,1255.130110,366.012760,8.488211e+03,504.374206,204.273215,3.234995e+04,583.728922,...,41.959372,42.364327,43.417402,46.200331,33.891907,34.644743,52.734118,55.138037,40.126155,40.561046
min,0.000000,7.000000,5.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,...,0.000000,0.000000,14.000000,14.000000,7.000000,7.000000,0.000000,0.000000,1.000000,1.000000
25%,22.000000,18.000000,33.000000,9.000000,25.000000,1.890000e+02,190.000000,191.000000,2.160000e+02,159.000000,...,34.000000,34.000000,32.000000,32.000000,46.000000,46.000000,19.000000,19.000000,40.000000,40.000000
50%,45.000000,52.000000,47.000000,40.000000,49.000000,3.460000e+02,335.000000,304.000000,4.770000e+02,278.000000,...,58.000000,58.000000,68.000000,68.000000,59.000000,59.000000,64.000000,64.000000,64.000000,64.000000
75%,83.000000,91.000000,76.000000,88.000000,86.000000,7.230000e+02,973.000000,442.000000,2.233000e+03,555.000000,...,82.000000,82.000000,92.000000,92.000000,78.000000,78.000000,98.000000,98.000000,87.000000,87.000000
max,101508.000000,144531.000000,80783.000000,103837.000000,40914.000000,1.439609e+06,2328.000000,3210.000000,8.994396e+06,34170.000000,...,500.000000,1000.000000,500.000000,1000.000000,500.000000,1000.000000,500.000000,1000.000000,500.000000,860.000000


In [143]:
bid_look_up.shape

(303925, 60)

In [144]:
# Add time preference
def time_preference(bid_look_up):
    for col in bid_look_up.columns:
        bid_look_up[col + '_calm'] = np.round(bid_look_up.loc[:,col] * np.linspace(0.8,1.2,bid_look_up.shape[0]))
        bid_look_up[col + '_aggre'] = np.round(bid_look_up.loc[:,col] * np.linspace(1.2,0.8,bid_look_up.shape[0]))
        
    return bid_look_up
        
bid_look_up = time_preference(bid_look_up)

In [146]:
bid_look_up.describe()

,log_reg_linbid,Damian_linbid,Stefan2_linbid,NN_linbid,ensemble1_linbid,log_reg_revlinbid,Damian_revlinbid,Stefan2_revlinbid,NN_revlinbid,ensemble1_revlinbid,...,Stefan2_ortb2_cap2_calm,Stefan2_ortb2_cap2_aggre,NN_ortb2_cap1_calm,NN_ortb2_cap1_aggre,NN_ortb2_cap2_calm,NN_ortb2_cap2_aggre,ensemble1_ortb2_cap1_calm,ensemble1_ortb2_cap1_aggre,ensemble1_ortb2_cap2_calm,ensemble1_ortb2_cap2_aggre
count,303925.000000,303925.000000,303925.000000,303925.000000,303925.000000,3.039250e+05,303925.000000,303925.000000,3.039250e+05,303925.000000,...,303925.000000,303925.000000,303925.000000,303925.000000,303925.000000,303925.000000,303925.000000,303925.000000,303925.000000,303925.000000
mean,85.999039,112.999355,86.000471,100.998523,84.999997,1.142597e+03,575.280309,335.563504,4.372314e+03,473.295247,...,67.049328,67.072715,66.187332,66.202754,66.388971,66.403320,67.638470,67.659732,67.669583,67.691898
std,487.782326,1526.586838,509.008216,1255.130110,366.012760,8.488211e+03,504.374206,204.273215,3.234995e+04,583.728922,...,35.605622,35.844675,53.602556,53.664326,56.015566,56.047281,41.082401,41.203381,41.497934,41.648434
min,0.000000,7.000000,5.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,...,6.000000,7.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,22.000000,18.000000,33.000000,9.000000,25.000000,1.890000e+02,190.000000,191.000000,2.160000e+02,159.000000,...,45.000000,45.000000,19.000000,19.000000,19.000000,19.000000,39.000000,39.000000,39.000000,39.000000
50%,45.000000,52.000000,47.000000,40.000000,49.000000,3.460000e+02,335.000000,304.000000,4.770000e+02,278.000000,...,59.000000,59.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000
75%,83.000000,91.000000,76.000000,88.000000,86.000000,7.230000e+02,973.000000,442.000000,2.233000e+03,555.000000,...,79.000000,79.000000,98.000000,98.000000,98.000000,98.000000,87.000000,87.000000,87.000000,87.000000
max,101508.000000,144531.000000,80783.000000,103837.000000,40914.000000,1.439609e+06,2328.000000,3210.000000,8.994396e+06,34170.000000,...,1076.000000,1199.000000,600.000000,600.000000,1184.000000,1199.000000,592.000000,599.000000,996.000000,1031.000000


In [145]:
bid_look_up.shape

(303925, 180)

In [147]:
# Save data file
## done bid_look_up.to_csv('proper_datasets/bid_look_up.csv')

### Load bids

In [4]:
bid_look_up = pd.read_csv('proper_datasets/bid_look_up.csv', index_col=0)

In [5]:
bid_look_up.shape

(303925, 180)

In [6]:
bid_look_up.head()

,log_reg_linbid,Damian_linbid,Stefan2_linbid,NN_linbid,ensemble1_linbid,log_reg_revlinbid,Damian_revlinbid,Stefan2_revlinbid,NN_revlinbid,ensemble1_revlinbid,...,Stefan2_ortb2_cap2_calm,Stefan2_ortb2_cap2_aggre,NN_ortb2_cap1_calm,NN_ortb2_cap1_aggre,NN_ortb2_cap2_calm,NN_ortb2_cap2_aggre,ensemble1_ortb2_cap1_calm,ensemble1_ortb2_cap1_aggre,ensemble1_ortb2_cap2_calm,ensemble1_ortb2_cap2_aggre
0,62.0,12.0,35.0,19.0,30.0,255.0,1396.0,416.0,982.0,459.0,...,38.0,58.0,31.0,47.0,31.0,47.0,37.0,55.0,37.0,55.0
1,24.0,15.0,25.0,2.0,10.0,661.0,1182.0,573.0,8811.0,1326.0,...,30.0,46.0,3.0,5.0,3.0,5.0,14.0,22.0,14.0,22.0
2,16.0,10.0,42.0,1.0,11.0,980.0,1749.0,344.0,13720.0,1276.0,...,44.0,66.0,2.0,4.0,2.0,4.0,15.0,23.0,15.0,23.0
3,24.0,12.0,33.0,3.0,14.0,647.0,1452.0,440.0,5973.0,1006.0,...,37.0,55.0,6.0,8.0,6.0,8.0,19.0,29.0,19.0,29.0
4,35.0,14.0,40.0,1.0,9.0,444.0,1193.0,361.0,21040.0,1513.0,...,42.0,64.0,2.0,2.0,2.0,2.0,13.0,19.0,13.0,19.0


In [8]:
############ FIT LINEAR BIDDING STRATEGY TO ALL 5 CTR ESTIMATORS ############

start = time.time()
clicks = []
budget = 6250 * 1000

for col in bid_look_up.iloc[:,:5].columns:
    print(col)
    bid = bid_look_up.loc[:,col] #base_bid * CTR_frac 
    
    df = val[val['payprice'] < bid][['click','payprice']]
    
    cost = df['payprice'].cumsum()
    #print(eligible)
    eligible = df[cost < budget]
    #print(eligible)
    
    click = eligible['click'].sum()
    wins = eligible.shape[0]
    CTR = click/wins
    costs = eligible['payprice'].sum()
    CPM = costs / wins / 1000
    CPC = costs / click / 1000
    
    print([click, wins, CTR, costs, CPM, CPC])
    
    clicks.append([click, wins, CTR, costs, CPM, CPC])

print(time.time()-start)    
clicks  

log_reg_linbid
[152, 126993, 0.001196916365468963, 6142918, 0.04837209924956494, 40.413934210526314]
Damian_linbid
[150, 132025, 0.0011361484567316796, 6206358, 0.04700896042416209, 41.37572]
Stefan2_linbid
[156, 134086, 0.001163432423966708, 5857102, 0.04368168190564265, 37.54552564102564]
NN_linbid
[164, 114938, 0.0014268562181349944, 5637105, 0.049044745862987, 34.37259146341463]
ensemble1_linbid
[166, 130810, 0.0012690161302652703, 6006462, 0.04591745279412889, 36.18350602409639]
0.34108614921569824


[[152,
  126993,
  0.001196916365468963,
  6142918,
  0.04837209924956494,
  40.413934210526314],
 [150, 132025, 0.0011361484567316796, 6206358, 0.04700896042416209, 41.37572],
 [156,
  134086,
  0.001163432423966708,
  5857102,
  0.04368168190564265,
  37.54552564102564],
 [164,
  114938,
  0.0014268562181349944,
  5637105,
  0.049044745862987,
  34.37259146341463],
 [166,
  130810,
  0.0012690161302652703,
  6006462,
  0.04591745279412889,
  36.18350602409639]]

In [10]:
new_res = pd.DataFrame(clicks)
new_res

,0,1,2,3,4,5
0,152,126993,0.001197,6142918,0.048372,40.413934
1,150,132025,0.001136,6206358,0.047009,41.375720
2,156,134086,0.001163,5857102,0.043682,37.545526
3,164,114938,0.001427,5637105,0.049045,34.372591
4,166,130810,0.001269,6006462,0.045917,36.183506


In [14]:
bid_look_up.columns[15:20]

Index(['log_reg_ortb2', 'Damian_ortb2', 'Stefan2_ortb2', 'NN_ortb2',
       'ensemble1_ortb2'],
      dtype='object')

In [15]:
############ FIT ORTB2 BIDDING STRATEGY TO ALL 5 CTR ESTIMATORS ############

start = time.time()
clicks = []
budget = 6250 * 1000

for col in bid_look_up.iloc[:,15:20].columns: # these indices are ORTB2 
    print(col)
    bid = bid_look_up.loc[:,col] 
    
    df = val[val['payprice'] < bid][['click','payprice']]
    
    cost = df['payprice'].cumsum()
    eligible = df[cost < budget]
    
    click = eligible['click'].sum()
    wins = eligible.shape[0]
    CTR = click/wins
    costs = eligible['payprice'].sum()
    CPM = costs / wins / 1000
    CPC = costs / click / 1000
    
    print([click, wins, CTR, costs, CPM, CPC])
    
    clicks.append([click, wins, CTR, costs, CPM, CPC])

print(time.time()-start)    
clicks  

log_reg_ortb2
[144, 141307, 0.00101905779614598, 6132826, 0.04340072324796365, 42.58906944444445]
Damian_ortb2
[145, 148365, 0.0009773194486570283, 6249980, 0.042125703501499676, 43.103310344827584]
Stefan2_ortb2
[147, 152161, 0.0009660819789565, 6168516, 0.04053940234357029, 41.96269387755102]
NN_ortb2
[154, 136614, 0.0011272636772219537, 6188671, 0.04530041576997965, 40.18617532467532]
ensemble1_ortb2
[160, 146939, 0.0010888872253111835, 6130721, 0.04172289861779378, 38.31700625]
0.37436532974243164


[[144,
  141307,
  0.00101905779614598,
  6132826,
  0.04340072324796365,
  42.58906944444445],
 [145,
  148365,
  0.0009773194486570283,
  6249980,
  0.042125703501499676,
  43.103310344827584],
 [147,
  152161,
  0.0009660819789565,
  6168516,
  0.04053940234357029,
  41.96269387755102],
 [154,
  136614,
  0.0011272636772219537,
  6188671,
  0.04530041576997965,
  40.18617532467532],
 [160,
  146939,
  0.0010888872253111835,
  6130721,
  0.04172289861779378,
  38.31700625]]

In [16]:
new_res2 = pd.DataFrame(clicks)
new_res2

,0,1,2,3,4,5
0,144,141307,0.001019,6132826,0.043401,42.589069
1,145,148365,0.000977,6249980,0.042126,43.103310
2,147,152161,0.000966,6168516,0.040539,41.962694
3,154,136614,0.001127,6188671,0.045300,40.186175
4,160,146939,0.001089,6130721,0.041723,38.317006


In [18]:
# AUC scores on validation set
from sklearn.metrics import roc_auc_score
for col in CTR_models_val.columns:
    print(col, roc_auc_score(val['click'], CTR_models_val.loc[:,col]))

log_reg 0.8050420111498808
Damian 0.8499709854827009
Stefan2 0.8455583453565674
NN 0.8860064422301417
ensemble1 0.8961361370083729
